In [2]:
import subprocess

from rdkit import Chem
from rdkit.Chem import rdMolAlign

import molclub
from molclub import xtb, conf_gen, utils

In [47]:
mol_1 = Chem.MolFromSmiles("OC(=O)C1=CC=CC=C1C1=CC=CC=N1")
mols_1 = conf_gen.etkdg(
    mol_1,
    prune_rms_thresh=0.5,
)
len(mols_1)

6

In [2]:
def basic_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    return_mol = Chem.rdchem.Mol(mol, quickCopy=True)
    return_mol.AddConformer(conformers[0])
    return return_mol

In [3]:
def no_prune_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    return_mol = Chem.rdchem.Mol(mol, quickCopy=True)
    return_mol.AddConformer(conformers[0])
    return return_mol

In [4]:
def mixed_prune_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        max_iters=10,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.25)
    
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )

    print(len(conformers))

    return_mol = Chem.rdchem.Mol(mol, quickCopy=True)
    return_mol.AddConformer(conformers[0])
    return return_mol

In [19]:
def xtb_ordered_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=1)

    mols = []
    for conformer in mol.GetConformers():
        temp_mol = Chem.rdchem.Mol(mol, quickCopy=True)
        temp_mol.AddConformer(conformer)
        mols.append(temp_mol)
    
    print(len(mols))
    
    mols, _ = xtb.order_conformers(mols)

    return mols[0]

In [20]:
def mixed_prune_xtb_ordered_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        max_iters=10,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.25)
    
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.5)

    mols = []
    for conformer in mol.GetConformers():
        temp_mol = Chem.rdchem.Mol(mol, quickCopy=True)
        temp_mol.AddConformer(conformer)
        mols.append(temp_mol)
    
    print(len(mols))
    
    mols, _ = xtb.order_conformers(mols)

    return mols[0]

In [4]:
def xtb_opt_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        max_iters=10,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.5)
    
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    mols = []
    for conformer in conformers:
        temp_mol = Chem.rdchem.Mol(mol, quickCopy=True)
        temp_mol.AddConformer(conformer)
        mols.append(temp_mol)
    mols, energies = xtb.order_conformers(mols)
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for m, e in zip(mols, energies):
        if e < 5:
            mol.AddConformer(m.GetConformer(), assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=1)
    
    tuples_mol_e = [xtb.optimize_xtb(mol) for mol in mols]
    mols = [tup[0] for tup in tuples_mol_e]
    energies = [tup[1] for tup in tuples_mol_e]

    energies, mols = zip(*sorted(zip(energies, mols)))

    return mols[0]

In [8]:
def xtb_opt_all_conf_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        max_iters=10,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.25)
    
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.5)

    mols = []
    for conformer in mol.GetConformers():
        temp_mol = Chem.rdchem.Mol(mol, quickCopy=True)
        temp_mol.AddConformer(conformer)
        mols.append(temp_mol)
    
    print(len(mols))

    mols = [xtb.optimize_xtb(mol)[0] for mol in mols]
    mols, _ = xtb.order_conformers(mols)

    return mols[0]

In [5]:
mol = Chem.MolFromSmiles('C[C@H]1CN([C@]12CCN(C2)C3=NC=NC4=C3C=CN4)C(=O)CC#N')

mols = []
for i in range(5):
    #                                           # time,         min,  max,  median
    # mols.append(basic_conf_gen(mol))          # 6.8 seconds,  0.4   2.4   2
    # mols.append(no_prune_conf_gen(mol))       # 8.1 seconds,  0.1   2.1   2
    # mols.append(mixed_prune_conf_gen(mol))    # 6.2 seconds,  0.1   2.2   2
    # mols.append(xtb_ordered_conf_gen(mol))    # 16.3 seconds, 0.5   2.2   2
    # mols.append(mixed_prune_xtb_ordered_conf_gen(mol)) # 6.0 s 0.2  2.3   2
    mols.append(xtb_opt_conf_gen(mol))        # 107 seconds,  0.02  0.8   0.5
    # mols.append(xtb_opt_all_conf_gen(mol))      # 330 seconds,  0.1   0.8  0.4

for mol_i in mols:
    for mol_j in mols:
        print(rdMolAlign.GetBestRMS(mol_i, mol_j))
    
    print('---------')

0.0
0.021257356640046948
0.8263921500921612
0.8281366216500258
0.12066318165918986
---------
0.02125735664096004
0.0
0.8318129922973702
0.8334094569309113
0.11458907769806148
---------
0.8263921501002844
0.8318129923050305
7.446945280485443e-08
0.05748875821587481
0.8349564004439892
---------
0.8281366216343039
0.8334094569313755
0.057488758215898925
0.0
0.8438189172067101
---------
0.12066318165933923
0.1145890776979405
0.8349564004443196
0.8438189172064735
0.0
---------


In [15]:
def xtb_opt_ensemble_gen(mol):
    mol = conf.etkdg_conf_gen(mol, prune_rms_thresh=-1, num_threads=4)
    conformers, energies = conf.mmff_optimization(
        mol,
        max_iters=10,
        num_threads=4,
    )
    min_e = min(energies)
    energies = [e - min_e for e in energies]
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for conformer, e in zip(conformers, energies):
        if e < 10:
            mol.AddConformer(conformer, assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.5)
    
    conformers, energies = conf.mmff_optimization(
        mol,
        num_threads=4,
    )
    mols = []
    for conformer in conformers:
        temp_mol = Chem.rdchem.Mol(mol, quickCopy=True)
        temp_mol.AddConformer(conformer)
        mols.append(temp_mol)
    mols, energies = xtb.order_conformers(mols)
    mol = Chem.rdchem.Mol(mol, quickCopy=True)
    for m, e in zip(mols, energies):
        if e < 5:
            mol.AddConformer(m.GetConformer(), assignId=True)
    mol = conf.prune_conformers(mol, prune_rms_thresh=0.5)
    
    tuples_mol_e = [xtb.optimize_xtb(mol) for mol in mols]
    mols = [tup[0] for tup in tuples_mol_e]
    energies = [tup[1] for tup in tuples_mol_e]

    energies, mols = zip(*sorted(zip(energies, mols)))

    return mols, energies

In [16]:
mol = Chem.MolFromSmiles('C[C@H]1CN([C@]12CCN(C2)C3=NC=NC4=C3C=CN4)C(=O)CC#N')

ensemble, energies = xtb_opt_ensemble_gen(mol)

In [17]:
len(ensemble)
display(energies)

(-40859.947990130255,
 -40859.90733281642,
 -40858.59670428443,
 -40858.590656915076,
 -40858.58143608056,
 -40858.570251144054,
 -40858.551246461946,
 -40858.51800367619,
 -40858.385044893046,
 -40857.179021923155,
 -40857.17377784165,
 -40857.08804846033,
 -40857.08652691026,
 -40857.075743000845,
 -40856.80615460788)

In [8]:
import py3Dmol

def display_3d_mol(mol: Chem.rdchem.Mol) -> None:
    """Use py3Dmol to visualize mol in 3D.
    Examples
    --------
    mol = Chem.MolFromSmiles('OCCCO')
    mol = get_low_energy_conformer(mol)
    display_3d_mol(mol)
    Parameters
    ----------
    mol: `rdkit.Chem.rdchem.Mol`
        The input RDKit mol object with an embedded 3D conformer.
    nonpolar_h: `bool`, default = False
        Whether or not to show nonpolar (C-H) hydrogens"""
    mol_block = ''
    mol_block = Chem.rdmolfiles.MolToMolBlock(mol, includeStereo=True)
    view = py3Dmol.view(data=mol_block,
                        style={'stick': {'colorscheme': 'grayCarbon'}})
    view.show()

In [18]:
for mol in ensemble:
    display_3d_mol(mol)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol